In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
np.random.seed(2)  

In [3]:
N_STATES = 6  # the length of the 1 dimensional world  环境的长度
ACTIONS = ['left', 'right'] # available actions   可选择的动作
EPSILON = 0.9 # greedy police  选择错误率(选择最大奖励的方向的概率)
ALPHA = 0.1 # learning rate 学习率
GAMMA = 0.9 # discount factor 衰减率
MAX_EPISODES = 13 # maximum episodes 训练次数
FRESH_TIME = 0.1 # fresh time for one move 刷新时间

In [4]:
# 创建Q table表格
def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))), # q_table initial values
        columns=actions, # actions's name  
    )
    return table
build_q_table(N_STATES, ACTIONS)

,left,right
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


In [5]:
# 选择一个方向
def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, : ]
    # 如果大于随机选择阀门 或者 初始值都为0 则进行随机选择
    if (np.random.uniform()>EPSILON) or ((state_actions==0).all()): # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS)
    else: # act greedy
        action_name = state_actions.idxmax() # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name

q_table = build_q_table(N_STATES, ACTIONS)
choose_action(1, q_table)

'right'

In [6]:
# 步数更新
def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right': # move right
        if S == N_STATES - 2: # terminate  终止
            S_ = 'terminal'
            R = 1
        else:
            S_  =  S+1
            R = 0
    else: # move left
        R = 0
        if S == 0:
            S_ = S # reach the wall  到达墙壁
        else:
            S_ = S - 1
    return S_, R 

In [7]:
# 更新环境
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1)+['T']  # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction))
#         print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)

In [8]:
def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A) # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal': 
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R  # next state is terminal
                is_terminated = True  # terminate this episode 
             
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state
            
            update_env(S, episode, step_counter+1)
            step_counter += 1
#         print(q_table)
    return q_table

In [9]:
q_table = rl()
print('\r\nQ-table:\n')
print(q_table)

Episode 1: total_steps = 37
Episode 2: total_steps = 22     
Episode 3: total_steps = 9      
Episode 4: total_steps = 5      
Episode 5: total_steps = 7      
Episode 6: total_steps = 5      
Episode 7: total_steps = 5      
Episode 8: total_steps = 5      
Episode 9: total_steps = 5      
Episode 10: total_steps = 5     
Episode 11: total_steps = 5     
Episode 12: total_steps = 7     
Episode 13: total_steps = 5     
                                
Q-table:

       left     right
0  0.000000  0.004320
1  0.000000  0.025005
2  0.000030  0.111241
3  0.000000  0.368750
4  0.027621  0.745813
5  0.000000  0.000000
